# SPEED workflow : Training on the spatial epigenomic data without prior information from single-cell data

Dataset: The E13 mouse embryo spatial CUT&Tag-RNA-seq dataset by Zhang et al ([here](https://doi.org/10.5281/zenodo.14948507))

In [1]:
import torch
print("Whether GPU is detected:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

Whether GPU is detected: True
CUDA version: 11.7


In [2]:
import SPEED
import scanpy as sc

adata_input_path = 'spCUT_Tag/tile_H3K27ac.h5ad'
adata_output_path = './H3K27ac_out'

load the spatial epigenomic data. 

## load the data

Load the spatial epigenomic data without the corresponding single-cell data. 

In [3]:
adata = sc.read(adata_input_path)

## Initialize the SPEED model

Initialize the model with spatial data.

`is_spatial` is set to `True` during the second stage of training on spatial data.

`k_degree` is the degree of spatial neighbor used for spatial relative position encoding. For data with a 50 μm resolution, k is defaulted to 5. For data with a 20 μm resolution, k is recommanded to 12.

`adata_sc` is set to `None` when training without prior information from single-cell data.

In [4]:
speed = SPEED.SPEED(adata,image=None, is_spatial=True,k_degree=12, adata_sc=None)

matrix ready...
use 0-1 matrix...
cell_features ready...
peak features ready...
Without single-cell reference


### Spliting training and validation sets.

`num_workers` is the number of subprocesses for data loading (default = 4).

`data_type` sets the input data format used by SPEED. SPEED will handle this format internally, so no external action is required from the user. For lower GPU memory and faster training, it is recommended to set `dense = False` (default) when training on GPU, and `dense = True` when training on CPU.

`batch_size_cell` and `batch_size_peak` are the batch sizes at the cell-level and peak-level. SPEED will choose automatically according to dataset size, but if the batch size is too large for your GPU, you can reduce it manually.

`split_ratio` sets the proportion of the validation set at both the cell level and peak level. (default = [1/6, 1/6])

In [5]:
speed.setup_data(num_workers=4)

batch_size_cell = 1024, batch_size_peak = 32768
split ready...
labels ready...
peak embedding is given
dataset ready...


### Build the neural network model for SPEED.

`emb_features` is the number of embedding features (default = 32).

`dropout_p` is the dropout probability of the model. For spatial data training, `dropout_p` is recommended to 0.4.

In [6]:
speed.build_model(emb_features=32,dropout_p=0.4)

## Train the SPEED model

`lr` is the learning rate. `device` specifies whether to train with GPU or CPU.

`epoch_num` is the maximum number of training epochs (default = 500). If no improvement is observed on the validation set within `epo_max` epochs, training is considered converged and will stop (default `epo_max=30`).

`alpha` represents the weight of the constraint on the similarity between peak embeddings of spatial data. The default value is 10. A larger `alpha` means the model relies more on single-cell prior information. 

`beta` represents the importance of image information for spot embedding. The default value is 1. A larger `beta` means the model relies more on image information.

In [7]:
speed.train(lr=1e-5, device='cuda:2')

Use spatial information...
Starting training...
trainset:  (7809, 245219) (204350, 9370)


  0%|          | 0/8 [00:00<?, ?it/s]/home/wangshuyan/workspace/project/SPEED/upload/environment/SPEED/SPEED/utils.py:104: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  return torch.sparse_csr_tensor(temp.indptr,temp.indices, temp.data, size=temp.shape)
100%|██████████| 2/2 [00:01<00:00,  1.47it/s]


Epoch[1/500], Loss: 0.86955, Val Loss: 0.86762, 


100%|██████████| 2/2 [00:01<00:00,  1.22it/s]


Epoch[2/500], Loss: 0.86895, Val Loss: 0.86786, 


...

Epoch[60/500], Loss: 0.82596, Val Loss: 0.84205, 


100%|██████████| 2/2 [00:01<00:00,  1.16it/s]


Epoch[61/500], Loss: 0.82562, Val Loss: 0.84239, 


100%|██████████| 2/2 [00:01<00:00,  1.16it/s]


Epoch[62/500], Loss: 0.82576, Val Loss: 0.84169, 
convinient


## Get the results

Use `SPEED.SPEED.get_embedding` to get the low-dimensional embedding.

The spot/cell embeddings will be stored in `adata.obsm['X_SPEED']`. The peak embeddings will be stored in `adata.varm['peak_SPEED']`

In [8]:
adata = speed.get_embedding(adata)

get cell/spot embedding...


100%|██████████| 5/5 [00:04<00:00,  1.13it/s]


get peak embedding...


100%|██████████| 5/5 [00:01<00:00,  4.31it/s]


get spatial embedding...


  0%|          | 0/5 [00:00<?, ?it/s]

(1875, 9370)
(1875, 9370)


100%|██████████| 5/5 [00:00<00:00, 24.61it/s]

(1875, 9370)
(1875, 9370)
(1870, 9370)
the shape of embedding: (9370, 32)


Use `SPEED.SPEED.get_denoise_result` to get the denoised matrix.

In [9]:
adata.X = speed.get_denoise_result()

In [10]:
adata = speed.binarize(adata)

100%|██████████| 245219/245219 [01:27<00:00, 2797.61it/s]


In [11]:
adata.write(f'H3K27ac_out/adata_speed_cpu.h5ad')

In [12]:
exit